# Wide ResNet
From: https://github.com/meliketoy/wide-resnet.pytorch/blob/master/networks/wide_resnet.py

Just altered the training script to take different save dir
The requirements.txt is incorrect, will just havr to guess

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from pathlib import Path
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import imshow
# from IPython.display import display, HTML
from collections import OrderedDict

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.nn import DataParallel

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, densenet121
from torchvision import datasets, models, transforms


%reload_ext autoreload
%autoreload 2
%matplotlib inline

PATH = Path('/media/rene/data')

# Add the src directory for functions
src_dir = Path.cwd().parent.parent / 'src'
print(src_dir)
sys.path.append(str(src_dir))
sys.path.append('/media/rene/code/wide-resnet.pytorch')


# import my functions:
from utils import make_cfiar10
from functions import*
from models import*
from networks import Wide_ResNet


device=torch.device("cuda:1")
print(torch.cuda.device_count())
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/home/rene/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rene/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


/media/rene/code/idc/src
2
True
0


## Find 4 best models

* Is there anything better about using their standard dataloaders rather than the way I did it?

In [7]:
# add directory for the wide resnet
sys.path.append('/media/rene/code/wide-resnet.pytorch')
from networks import *

save_path = '/media/rene/data/cifar-10-batches-py/wide-RN-models'
num_workers = 4
batch_size=32
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                   valid_name='valid')

criterion = nn.CrossEntropyLoss()
results = {}

for i in range(6):
    model_name = 'wideRN'+str(i)+'.t7'
    model = Wide_ResNet(28, 20, 0, 10)
    device="cuda:1"
    model = model.to(device).eval()
    
    # load the saved weights
    model_path = os.path.join(save_path, model_name)
    print(model_path)
    model.load_state_dict(torch.load(model_path)['net'].state_dict())

    print('Testing model : ', i)
    valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
    results[model_name] = [valid_acc]
    
f = open(os.path.join(save_path, "wideRN_al68.pkl"),"wb")
pickle.dump(results,f)
f.close()

Files already downloaded and verified
Files already downloaded and verified
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN0.t7
Testing model :  0
Loss: 0.0046 Acc: 0.9601
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN1.t7
Testing model :  1
Loss: 0.0046 Acc: 0.9600
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN2.t7
Testing model :  2
Loss: 0.0046 Acc: 0.9614
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN3.t7
Testing model :  3
Loss: 0.0048 Acc: 0.9600
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN4.t7
Testing model :  4
Loss: 0.0046 Acc: 0.9600
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN5.t7
Testing model :  5
Loss: 0.0045 Acc: 0.9612


In [8]:
import operator

results2 = {key:value[0] for (key,value) in results.items()}
sorted_x = sorted(results2.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_x)

[('wideRN2.t7', 0.9614), ('wideRN5.t7', 0.9612), ('wideRN0.t7', 0.9601), ('wideRN1.t7', 0.96), ('wideRN3.t7', 0.96), ('wideRN4.t7', 0.96)]


# Fusion Model

In [12]:
save_path =  Path(PATH / 'cifar-10-batches-py' / 'wide-RN-models')

model_name_list = ['wideRN2.t7', 'wideRN5.t7', 'wideRN0.t7', 'wideRN1.t7']
num_workers = 6
batch_size = 8
criterion = nn.CrossEntropyLoss()

dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                   valid_name='valid')

pretrained_model_list = []
for i, model_name in enumerate(model_name_list):
    model = Wide_ResNet(28, 20, 0, 10)
    model = model.to(device)
    
    # load the saved weights
    model_path = os.path.join(save_path, model_name)
    print(model_path)
    model.load_state_dict(torch.load(model_path)['net'].state_dict())
    pretrained_model_list.append(model)
    
model = Fusion2(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_WRN_1')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
print('Fusion2_WRN_1: ', valid_loss, valid_acc)

model = Fusion2(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_WRN_2')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Fusion2_WRN_2: ', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN2.t7
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN5.t7
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN0.t7
| Wide-Resnet 28x20
/media/rene/data/cifar-10-batches-py/wide-RN-models/wideRN1.t7
Loss: 0.0227 Acc: 0.9687
Fusion2_WRN_1:  0.02274664867669344 0.9687
Loss: 0.0180 Acc: 0.9672
Fusion2_WRN_2:  0.017988840979337693 0.9672


# Different Wide ResNet
https://github.com/xternalz/WideResNet-pytorch.git

source WM/bin/activate
python train.py --dataset cifar10 --layers 28 --widen-factor 20
(just delete the tensoerboard logger, add print statement to see where saving)

In [12]:
# add directory for the wide resnet
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

PATH = '/media/rene/data'
num_workers = 4
batch_size=32

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])
testset = torchvision.datasets.CIFAR10(root=PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


criterion = nn.CrossEntropyLoss()

model_name = 'model_best.pth.tar'
model = WideResNet(28, 10, 20, dropRate=0)
model = model.to(device)

results = {}
for i in range(6):
    save_path = '/media/rene/code/WideResNet-pytorch/runs/WideResNet-28-10_'+str(i)
    print('loading: ', save_path)

    # original saved file with DataParallel
    state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
    
    # Different config if it was trainied with DataParallel
    if i ==0:
        print('model was saved as DataParallel')
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        # load params
        model.load_state_dict(new_state_dict)
    else:
        model.load_state_dict(state_dict)
        
    valid_loss, valid_acc = eval_model(model, testloader, len(testset), criterion, device=device)
    print('valid_acc:', valid_acc)

    results[str(i)] = [valid_acc]
    
print(results)

Files already downloaded and verified
loading:  /media/rene/code/WideResNet-pytorch/runs/WideResNet-28-10_0
model was saved as DataParallel
Loss: 0.0044 Acc: 0.9650
valid_acc: 0.965
loading:  /media/rene/code/WideResNet-pytorch/runs/WideResNet-28-10_1
Loss: 0.0048 Acc: 0.9622
valid_acc: 0.9622
loading:  /media/rene/code/WideResNet-pytorch/runs/WideResNet-28-10_2
Loss: 0.0045 Acc: 0.9622
valid_acc: 0.9622
loading:  /media/rene/code/WideResNet-pytorch/runs/WideResNet-28-10_3
Loss: 0.0049 Acc: 0.9620
valid_acc: 0.962
loading:  /media/rene/code/WideResNet-pytorch/runs/WideResNet-28-10_4
Loss: 0.0045 Acc: 0.9641
valid_acc: 0.9641
loading:  /media/rene/code/WideResNet-pytorch/runs/WideResNet-28-10_5
Loss: 0.0044 Acc: 0.9636
valid_acc: 0.9636
{'0': [0.965], '1': [0.9622], '2': [0.9622], '3': [0.962], '4': [0.9641], '5': [0.9636]}


## Test Fusion

In [2]:
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

PATH = Path('/media/rene/data/')
save_path = Path('/media/rene/code/WideResNet-pytorch/runs')
model_name_list = ['WideResNet-28-10_0/model_best.pth.tar', 'WideResNet-28-10_1/model_best.pth.tar', 'WideResNet-28-10_2/model_best.pth.tar', 
                   'WideResNet-28-10_3/model_best.pth.tar', 'WideResNet-28-10_4/model_best.pth.tar', 'WideResNet-28-10_5/model_best.pth.tar']
batch_size = 4
num_workers = 4
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid')

pretrained_model_list = []
# First trained model was with DATA PARALLEL
model = WideResNet(28, 10, 20)
model = model.to(device)
# state_dict = torch.load(os.path.join(save_path, 'WideResNet-28-10_0/model_best.pth.tar'))['state_dict']

# # create new OrderedDict that does not contain `module.`
# new_state_dict = OrderedDict()
# for k, v in state_dict.items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# model.load_state_dict(new_state_dict)

# pretrained_model_list.append(model)

# get all the models
for i, model_name in enumerate(model_name_list[2:]):
    print('------------loading model: ', model_name)
    model = WideResNet(28, 10, 20)
    model = model.to(device)

    # original saved file with DataParallel
    state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
    model.load_state_dict(state_dict)
    pretrained_model_list.append(model)

model = Fusion2(pretrained_model_list, num_input=60, num_output=10).to(device)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified
------------loading model:  WideResNet-28-10_2/model_best.pth.tar
------------loading model:  WideResNet-28-10_3/model_best.pth.tar
------------loading model:  WideResNet-28-10_4/model_best.pth.tar
------------loading model:  WideResNet-28-10_5/model_best.pth.tar


In [3]:
model.load_state_dict(torch.load(os.path.join('/media/rene/code/WideResNet-pytorch/runs/Fusion6_WRN_1')))

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
print('Fusion6_WRN_1: ', valid_loss, valid_acc)

Loss: 0.0525 Acc: 0.9558
Fusion6_WRN_1:  0.05246691736280918 0.9558


In [5]:
model.load_state_dict(torch.load(os.path.join('/media/rene/code/WideResNet-pytorch/runs/Fusion4_WRN_1')))

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
print('Fusion4_WRN_1: ', valid_loss, valid_acc)

Loss: 0.0461 Acc: 0.9620
Fusion4_WRN_1:  0.04608403277993202 0.962


# LAST ATTEMPT
* 4 networks
* Only fine tuning the last added layer
* ~100 epochs, so should've worked
* Had trained using the wrong normalization, comparing the two below:

In [3]:
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

with torch.cuda.device(1):
    PATH = Path('/media/rene/data/')
    save_path = Path('/media/rene/code/WideResNet-pytorch/runs')
    model_name_list = ['WideResNet-28-10_0/model_best.pth.tar', 'WideResNet-28-10_1/model_best.pth.tar', 'WideResNet-28-10_2/model_best.pth.tar', 
                       'WideResNet-28-10_3/model_best.pth.tar', 'WideResNet-28-10_4/model_best.pth.tar', 'WideResNet-28-10_5/model_best.pth.tar']
    batch_size = 4
    num_workers = 4

    dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                        valid_name='valid', transformation=None)

    pretrained_model_list = []
    for i, model_name in enumerate(model_name_list[2:]):
        print('------------loading model: ', model_name)
        model = WideResNet(28, 10, 20)
        model = model.to(device)

        # original saved file with DataParallel
        state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
        model.load_state_dict(state_dict)
        pretrained_model_list.append(model)

    model = Fusion2(pretrained_model_list, num_input=60, num_output=10).to(device)
    criterion = nn.CrossEntropyLoss()


    model.load_state_dict(torch.load(os.path.join('/media/rene/code/WideResNet-pytorch/runs/fusionWRN_last4_1')))

    valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
    print('fusionWRN_last4_1: ', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
------------loading model:  WideResNet-28-10_2/model_best.pth.tar
------------loading model:  WideResNet-28-10_3/model_best.pth.tar
------------loading model:  WideResNet-28-10_4/model_best.pth.tar
------------loading model:  WideResNet-28-10_5/model_best.pth.tar
Loss: 0.0434 Acc: 0.9611
fusionWRN_last4_1:  0.043354870277643204 0.9611


In [2]:
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

PATH = Path('/media/rene/data/')
save_path = Path('/media/rene/code/WideResNet-pytorch/runs')
model_name_list = ['WideResNet-28-10_0/model_best.pth.tar', 'WideResNet-28-10_1/model_best.pth.tar', 'WideResNet-28-10_2/model_best.pth.tar', 
                   'WideResNet-28-10_3/model_best.pth.tar', 'WideResNet-28-10_4/model_best.pth.tar', 'WideResNet-28-10_5/model_best.pth.tar']
batch_size = 4
num_workers = 4

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid', transformation=transform_test)

pretrained_model_list = []
for i, model_name in enumerate(model_name_list[2:]):
    print('------------loading model: ', model_name)
    model = WideResNet(28, 10, 20)
    model = model.to(device)

    # original saved file with DataParallel
    state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
    model.load_state_dict(state_dict)
    pretrained_model_list.append(model)

model = Fusion2(pretrained_model_list, num_input=60, num_output=10).to(device)
criterion = nn.CrossEntropyLoss()


model.load_state_dict(torch.load(os.path.join('/media/rene/code/WideResNet-pytorch/runs/fusionWRN_last4_1')))

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
print('fusionWRN_last4_1: ', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
------------loading model:  WideResNet-28-10_2/model_best.pth.tar
------------loading model:  WideResNet-28-10_3/model_best.pth.tar
------------loading model:  WideResNet-28-10_4/model_best.pth.tar
------------loading model:  WideResNet-28-10_5/model_best.pth.tar
Loss: 0.0351 Acc: 0.9686
fusionWRN_last4_1:  0.03512242842018604 0.9686


## Average predictions of N networks

In [12]:
# get all the models:
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

save_path = Path('/media/rene/code/WideResNet-pytorch/runs')
model_name_list = ['WideResNet-28-10_0/model_best.pth.tar', 'WideResNet-28-10_1/model_best.pth.tar', 'WideResNet-28-10_2/model_best.pth.tar', 
                   'WideResNet-28-10_3/model_best.pth.tar', 'WideResNet-28-10_4/model_best.pth.tar', 'WideResNet-28-10_5/model_best.pth.tar']
batch_size = 4
num_workers = 4


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid', transformation=transform_test)

pretrained_model_list = []

# One with with DATA PARALLEL:
model = WideResNet(28, 10, 20)
model = model.to(device)
state_dict = torch.load(os.path.join(save_path, 'WideResNet-28-10_0/model_best.pth.tar'))['state_dict']

# create new OrderedDict that does not contain `module.`
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
pretrained_model_list.append(model)

# get all the models
for i, model_name in enumerate(model_name_list[1:]):
    print('------------loading model: ', model_name)
    model = WideResNet(28, 10, 20)
    model = model.to(device)

    # original saved file with DataParallel
    state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
    model.load_state_dict(state_dict)
    pretrained_model_list.append(model)

Files already downloaded and verified
Files already downloaded and verified
------------loading model:  WideResNet-28-10_1/model_best.pth.tar
------------loading model:  WideResNet-28-10_2/model_best.pth.tar
------------loading model:  WideResNet-28-10_3/model_best.pth.tar
------------loading model:  WideResNet-28-10_4/model_best.pth.tar
------------loading model:  WideResNet-28-10_5/model_best.pth.tar


In [16]:
def get_model_list_preds(model_list, dataloader):
    results = pd.DataFrame()

    for (inputs, labels) in dataloader:
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())
        
        output_dict = {}
        output_dict['label'] = labels.item()
        
        for i, model in enumerate(model_list):
            model.eval()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            preds = preds.item()
            name = 'model_'+str(i)
            output_dict[name] = preds
            del outputs 
        results = results.append(output_dict, ignore_index=True)
    return results

In [17]:
batch_size = 1
num_workers = 4
PATH = Path('/media/rene/data/')

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid', transforms=transform_test)

results = get_model_list_preds(pretrained_model_list, dataloaders['valid'])

Files already downloaded and verified
Files already downloaded and verified


In [19]:
acc = len(results[results['label']==results['model_0']])/len(results)
print(acc)
acc = len(results[results['label']==results['model_1']])/len(results)
print(acc)
acc = len(results[results['label']==results['model_2']])/len(results)
print(acc)
acc = len(results[results['label']==results['model_3']])/len(results)
print(acc)
acc = len(results[results['label']==results['model_4']])/len(results)
print(acc)
acc = len(results[results['label']==results['model_5']])/len(results)
print(acc)

0.965
0.9622
0.9622
0.962
0.9641
0.9636


In [3]:
def eval_model(model, dataloader, dataset_size, criterion, device="cuda:0"):
    model.train(False)  # Set model to evaluate mode
    model.eval()

    running_loss = 0.0
    running_corrects = 0
    total = 0

    # Iterate over data.
    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)   

        # forward
        outputs = model(inputs)

        # for nets that have multiple outputs such as inception
        if isinstance(outputs, tuple):
            loss = sum((criterion(o,labels) for o in outputs))
        else:
            loss = criterion(outputs, labels)
        
        _, preds = outputs.max(1)
        running_loss += loss.item()
        running_corrects += preds.eq(labels).sum().item()
        total += labels.size(0)    
    del _, loss, outputs, preds, labels, inputs

    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects / dataset_size
    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    return epoch_loss, epoch_acc

In [9]:
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

batch_size = 1
num_workers = 4
PATH = Path('/media/rene/data/')

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])

dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid', transforms=transform_test)
model = WideResNet(28, 10, 20)
model = model.to(device)

save_path = '/media/rene/code/WideResNet-pytorch/runs/'
model_name = 'WideResNet-28-10_2/model_best.pth.tar'

state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
model.load_state_dict(state_dict)

criterion = nn.CrossEntropyLoss()
valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
print('WideResNet: ', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
Loss: 0.1442 Acc: 0.9622
WideResNet:  0.1441868129134178 0.9622


# Final - using proper normalization

In [ ]:
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

with torch.cuda.device(1):
    PATH = Path('/media/rene/data/')
    save_path = Path('/media/rene/code/WideResNet-pytorch/runs')
    model_name_list = ['WideResNet-28-10_0/model_best.pth.tar', 'WideResNet-28-10_1/model_best.pth.tar', 'WideResNet-28-10_2/model_best.pth.tar', 
                       'WideResNet-28-10_3/model_best.pth.tar', 'WideResNet-28-10_4/model_best.pth.tar', 'WideResNet-28-10_5/model_best.pth.tar']
    batch_size = 4
    num_workers = 4

    dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                        valid_name='valid', transformation=None)

    pretrained_model_list = []
    for i, model_name in enumerate(model_name_list[1:3]):
        print('------------loading model: ', model_name)
        model = WideResNet(28, 10, 20)
        model = model.to(device)

        # original saved file with DataParallel
        state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
        model.load_state_dict(state_dict)
        pretrained_model_list.append(model)

    model = Fusion2(pretrained_model_list, num_input=60, num_output=10).to(device)
    criterion = nn.CrossEntropyLoss()


    model.load_state_dict(torch.load(os.path.join('/media/rene/code/WideResNet-pytorch/runs/fusionWRN_last4_1')))

    valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
    print('fusionWRN_last4_1: ', valid_loss, valid_acc)

In [2]:
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

with torch.cuda.device(1):
    PATH = Path('/media/rene/data/')
    save_path = Path('/media/rene/code/WideResNet-pytorch/runs')
    model_name_list = ['WideResNet-28-10_0/model_best.pth.tar', 'WideResNet-28-10_1/model_best.pth.tar', 'WideResNet-28-10_2/model_best.pth.tar', 
                       'WideResNet-28-10_3/model_best.pth.tar', 'WideResNet-28-10_4/model_best.pth.tar', 'WideResNet-28-10_5/model_best.pth.tar']
    batch_size = 16
    num_workers = 4


    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])
    ])
    dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                        valid_name='valid', transformation=transform_test)

    pretrained_model_list = []
    # First trained model was with DATA PARALLEL
    model = WideResNet(28, 10, 20)
    model = model.to(device)

    state_dict = torch.load(os.path.join(save_path, 'WideResNet-28-10_0/model_best.pth.tar'))['state_dict']

    # create new OrderedDict that does not contain `module.`
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    pretrained_model_list.append(model)

    # get all the models
    for i, model_name in enumerate(model_name_list[1:3]):
        print('------------loading model: ', model_name)
        model = WideResNet(28, 10, 20)
        model = model.to(device)

        # original saved file with DataParallel
        state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
        model.load_state_dict(state_dict)
        pretrained_model_list.append(model)

    criterion = nn.CrossEntropyLoss()
    model = Fusion3(pretrained_model_list, num_input=30, num_output=10).to(device)
    
    model.load_state_dict(torch.load(os.path.join('/media/rene/code/WideResNet-pytorch/runs/fusionWRN_last3_1')))
    valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
    print('fusionWRN_last3_1', valid_loss, valid_acc)
    
    model.load_state_dict(torch.load(os.path.join('/media/rene/code/WideResNet-pytorch/runs/fusionWRN_last3_2')))
    valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
    print('fusionWRN_last3_2', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
------------loading model:  WideResNet-28-10_1/model_best.pth.tar
------------loading model:  WideResNet-28-10_2/model_best.pth.tar
Loss: 0.0082 Acc: 0.9697
fusionWRN_last3_1 0.008204502047598362 0.9697
Loss: 0.0083 Acc: 0.9699
fusionWRN_last3_2 0.008345628140866757 0.9699


In [11]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
get_n_params(model)

437447536

In [12]:
get_n_params(model.model1)

145815514